# Detección de arrtimias cardiacas: Asistolia, Bradicardia extrema, Taquicardia extrema, taquicardia ventricular, fibrilación ventricular

Demo para realizar la detección de comportamientos anómalos en el ritmo cardiaco.

In [1]:
!pip install wfdb
!pip install neurokit2 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from ipywidgets import interact
import wfdb
import os
import sys  

In [3]:
# https://drive.google.com/file/d/13SB6ynb-QlCNq6I5GFU3YwIW0hJ371m3/view?usp=sharing

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=13SB6ynb-QlCNq6I5GFU3YwIW0hJ371m3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=13SB6ynb-QlCNq6I5GFU3YwIW0hJ371m3" -O "training.zip" && rm -rf /tmp/cookies.txt

--2022-11-01 15:27:15--  https://docs.google.com/uc?export=download&confirm=t&id=13SB6ynb-QlCNq6I5GFU3YwIW0hJ371m3
Resolving docs.google.com (docs.google.com)... 142.251.111.102, 142.251.111.101, 142.251.111.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.111.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-7g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/l24k6hjd05pcgaltgr985qe6f5p0obv6/1667316375000/08333744685695600646/*/13SB6ynb-QlCNq6I5GFU3YwIW0hJ371m3?e=download&uuid=4bbf6e7e-5f1d-4e02-8bbb-eb6d40332152 [following]
--2022-11-01 15:27:15--  https://doc-00-7g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/l24k6hjd05pcgaltgr985qe6f5p0obv6/1667316375000/08333744685695600646/*/13SB6ynb-QlCNq6I5GFU3YwIW0hJ371m3?e=download&uuid=4bbf6e7e-5f1d-4e02-8bbb-eb6d40332152
Resolving doc-00-7g-docs.googleusercontent.com (doc-00-7g-docs.googleusercontent.com)... 142.251.163.

In [4]:
!unzip training.zip -d training

Archive:  training.zip
replace training/training/RECORDS? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [8]:
# Funciones para análisis de anomalías
!rm -rf Humath
!git clone https://github.com/juliangilg/Humath
sys.path.append('Humath/Functions')
from AnomalyDetection_ECG import anomalydetection_ecg

Cloning into 'Humath'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 54 (delta 24), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), done.


In [9]:
import scipy.io as sio
import neurokit2 as nk
import wfdb
from wfdb import processing
# from window_slider import Slider
import numpy as np
import sys
import matplotlib.pyplot as plt
from scipy import signal
from scipy import stats as st
from scipy.signal import butter,filtfilt

In [10]:
Fs = 250 # frecuencia de muestreo.  
plt.rcParams['figure.figsize'] = 30, 10

In [11]:
path = './training/training/'
signals_count = []
signals_ = []
Label = []
list_files = [f for f in os.listdir(path) if f.endswith('.hea')]
Fs = 250 # frecuencia de muestreo.  
for nfile in list_files:
  signals, fields = wfdb.rdsamp(os.path.join(path, nfile[:-4]))
  typeSig_n = fields['sig_name']
  signals_count.append(typeSig_n)
  signals_.append(signals)
  Label.append([fields['comments'][0],fields['comments'][1]])

In [12]:
for k in [529, 621, 341,415,23, 43, 738, 710, 616, 73, 159, 203, 224, 232, 81]: 
  aux_ECG = [nam for nam in signals_count[k] if (nam != 'RESP') & (nam != 'PLETH') & (nam != 'ABP')][0]
  tvent = 0.8 # duración de la ventana en segundos
  t = np.linspace(0,20,Fs*20)
  AnoDet = anomalydetection_ecg(Fs = Fs, tenvt = tvent)
  Asy = [11,14,16,18,33,37,45,54,55,56]
  idx = signals_count[k].index(aux_ECG)
  x =  signals_[k][70000:75000,idx]
  corru = AnoDet.fit_transform(x)
  corru[corru==0] = np.NaN
   
 

  max_ = x.max()
  if (np.isnan(max_)) | (abs(max_) <0.01):
    max_ = 1
  min_ = x.min()
  if (np.isnan(min_)) | (abs(min_) <0.01):
    min_ = -1
  if abs(max_ - min_) < 0.5:
    max_ += 0.5
    min_ -= 0.5

 
  plt.show()  
  plt.plot(t,x)
  plt.title(str(k) + "," + list_files[k][:-4] + ":" + Label[k][0] + ':'+ Label[k][1])
  # k += 1
  # print(max_, min_)
  plt.fill_between(t, max_*corru, min_*corru, alpha=0.3, color='gray')
  plt.show()
  


  
  


Output hidden; open in https://colab.research.google.com to view.